# Modello di deep learning per predizione dropout scolastico su dati INVALSI
Progetto del corso di **Intelligenza Artificiale**, A.A. 2020/2021

**LM Informatica**, **Alma Mater Studiorum - Università di Bologna**

Realizzato da:
- Marco Ferrati, matr. 983546
- Michele Perlino, matr. 983733
- Tommaso Azzalin, matr. 985911

Per aprire il notebook corrente sulla piattaforma Google Colab cliccare su [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MickPerl/MachineLearningProject/blob/main/AzzalinFerratiPerlino-INVALSI.ipynb)

# Setup dell'ambiente

## Installazione delle librerie necessarie

In [2]:
!pip3 install --no-cache-dir -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


## Import delle librerie fondamentali per l'analisi dei dati

In [3]:
import re
from beautifultable import BeautifulTable

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import IntegerLookup
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow.keras.callbacks import EarlyStopping

import src.config as cfg
from src.mapping_domande_ambiti_processi import MAPPING_DOMANDE_AMBITI_PROCESSI
from src.column_converters import COLUMN_CONVERTERS

## Import del dataset originale

In [4]:
"""
Eseguire per: analisi dataset
"""
original_dataset = pd.read_csv(cfg.ORIGINAL_DATASET, sep=';', converters=COLUMN_CONVERTERS)

# EDA del dataset originale
Il codice che segue realizza il processo EDA, acronimo che sta per *exploratory data analysis*, al fine di scandagliare i punti di attenzione del dataset a disposizione.

In [5]:
original_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342226 entries, 0 to 342225
Columns: 104 entries, Unnamed: 0 to DROPOUT
dtypes: bool(52), float64(16), int64(11), object(25)
memory usage: 152.7+ MB


Come restituito dalla funzione `info()`, il dataset presenta $342226$ righe e $104$ colonne.

In [6]:
original_dataset.columns

Index(['Unnamed: 0', 'CODICE_SCUOLA', 'CODICE_PLESSO', 'CODICE_CLASSE',
       'macrotipologia', 'campione', 'livello', 'prog', 'CODICE_STUDENTE',
       'sesso',
       ...
       'PesoScuola', 'PesoTotale_Matematica', 'WLE_MAT', 'WLE_MAT_200',
       'WLE_MAT_200_CORR', 'pu_ma_no_corr', 'n_stud_prev', 'n_classi_prev',
       'LIVELLI', 'DROPOUT'],
      dtype='object', length=104)

Il dataset contiene svariate informazioni relative a studenti che hanno ultimato il ciclo di studi delle superiori: l'obiettivo del progetto è la progettazione e implementazione di un classificatore capace di predire, sulla base dei risultati conseguiti al test INVALSI di terza media, quali studenti registreranno un *dropout*.\
Il concetto di *dropout* può essere declinato su due livelli:
- implicito: si dice che lo studente ha registrato un dropout implicito nel caso in cui non abbia acquisito le minime conoscenze e competenze, per cui presenta lacune formative;
- esplicito: si dice che lo studente ha registrato un dropout esplicito nel caso in cui non abbia conseguito il diploma.

## Ricerca di colonne con alte percentuali di valori nulli

In [7]:
print("Columns with high null values percentages:")

table = BeautifulTable()
table.columns.header = ["", "Type","Ratio null values"]

for col in original_dataset.columns :
    ratio_null_values = original_dataset[col].isnull().mean().round(3)
    if ratio_null_values > 0:
        table.rows.append([col, str(original_dataset[col].dtypes), ratio_null_values]) 
        
table.columns.alignment = BeautifulTable.ALIGN_LEFT
table.set_style(BeautifulTable.STYLE_SEPARATED)
table.rows.sort('Ratio null values')
print(table)

Columns with high null values percentages:
+=======================+=========+===================+
|                       | Type    | Ratio null values |
+=======================+=========+===================+
| voto_orale_ita        | float64 | 0.113             |
+-----------------------+---------+-------------------+
| voto_orale_mat        | float64 | 0.114             |
+-----------------------+---------+-------------------+
| voto_scritto_ita      | float64 | 0.683             |
+-----------------------+---------+-------------------+
| voto_scritto_mat      | float64 | 0.683             |
+-----------------------+---------+-------------------+
| PesoClasse            | float64 | 0.946             |
+-----------------------+---------+-------------------+
| PesoScuola            | float64 | 0.946             |
+-----------------------+---------+-------------------+
| PesoTotale_Matematica | float64 | 0.946             |
+-----------------------+---------+-------------------+
| cod

In [8]:
columns_high_ratio_null_values = ["codice_orario", "PesoClasse", "PesoScuola", "PesoTotale_Matematica"]
columns_lower_ratio_null_values = [
    "voto_scritto_ita",  # 0.683
    "voto_scritto_mat",  # 0.113
    "voto_orale_ita",  # 0.683
    "voto_orale_mat"  # 0.114
]

## Ricerca colonne con un numero basso di valori distinti 

In [9]:
print("Columns with unique values:")

table = BeautifulTable()
table.columns.header = ["", "Ratio distinct values"]

for col in original_dataset.columns:
    ratio_unique_vals = round(original_dataset[col].nunique() / len(original_dataset), 3)
    if ratio_unique_vals  > 0.1 :
        table.rows.append([col, ratio_unique_vals]) 

table.columns.alignment = BeautifulTable.ALIGN_LEFT
table.set_style(BeautifulTable.STYLE_SEPARATED)
table.rows.sort('Ratio distinct values')
print(table)

Columns with unique values:
+==================+=======================+
|                  | Ratio distinct values |
+==================+=======================+
| WLE_MAT_200_CORR | 0.464                 |
+------------------+-----------------------+
| pu_ma_no_corr    | 0.494                 |
+------------------+-----------------------+
| Unnamed: 0       | 1.0                   |
+------------------+-----------------------+
| CODICE_STUDENTE  | 1.0                   |
+------------------+-----------------------+


Come era prevedibile, `Unnamed: 0` e `CODICE_STUDENTE` presentano una proporzione di valori distinti sul totale delle righe pari a 1, in quanto trattasi di indici/codici che identificano univocamente gli studenti. Queste due colonne potranno essere eliminate, dato che non portano alcuna informazione che possa aiutare a ravvisare relazioni tra gli studenti.

In [10]:
columns_with_unique_values = ["Unnamed: 0", "CODICE_STUDENTE"]

## Ricerca colonne con un singolo valore

In [11]:
print("Columns with just one value:")
for col in original_dataset.columns:
    unique_vals = original_dataset[col].nunique()
    if unique_vals == 1:
        print(col)

Columns with just one value:
macrotipologia
livello


Similmente, queste colonne possono essere eliminate in quanto non distinguono in alcuna maniera gli studenti.

In [12]:
columns_with_just_one_value = ["macrotipologia", "livello"]

## Rimozione delle colonne superflue

In [13]:
"""
Eseguire per: analisi dataset
"""
cleaned_original_dataset: pd.DataFrame = original_dataset.drop(
    columns_high_ratio_null_values + 
    columns_with_unique_values + 
    columns_with_just_one_value, 
    axis=1
)
cleaned_original_dataset.to_csv(cfg.CLEANED_DATASET)

In [14]:
"""
Eseguire per: analisi dataset
Attenzione: inutile eseguire se si è eseguita la precedente cella.
"""
cleaned_original_dataset = pd.read_csv(cfg.CLEANED_DATASET)

C:\Users\miche\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
if "Unnamed: 0" in cleaned_original_dataset.columns:
    cleaned_original_dataset.drop("Unnamed: 0", axis=1, inplace=True)

## Generalizzazione dataset per supportare altre coorti di studenti
Addestrando la rete con le domande specifiche di un certo test INVALSI non si ottiene un classificatore riutilizzabile per coorti successive. Pertanto, abbiamo pensato di mappare le feature inerenti alle domande in uno spazio più generico che permetta di cogliere la semantica delle domande piuttosto che la loro rappresentazione letterale; mediante le griglie di correzione fornite ai docenti, abbiamo notato che ogni domanda è caratterizzato da un ambito e da un processo: questa corrispondenza ha ispirato la trasformazione dello spazio di rappresentazione delle domande che riportiamo di seguito. 

Nel file `mapping_domande_ambiti_processi` posizionato nella cartella `src` abbiamo creato un dizionario le cui chiavi sono le domande e i valori gli ambiti e i processi corrispondenti. Dopo averlo importato nel notebook corrente, abbiamo estratto i distinti ambiti e processi per poi calcolarne il numero di domande che caratterizzano.

In [16]:
list_ambiti_processi = [AP for val in MAPPING_DOMANDE_AMBITI_PROCESSI.values() for AP in val]
ambiti_processi = set(list_ambiti_processi)
conteggio_ambiti_processi = {AP: list_ambiti_processi.count(AP) for AP in ambiti_processi}   

Abbiamo proceduto con l'aggiungere al dataset originale colonne recanti il nome degli ambiti e processi, inizializzate a 0 per tutte le righe.

In [17]:
"""
Eseguire per: analisi dataset
"""
dataset_with_ambiti_processi = cleaned_original_dataset.copy()
for AP in ambiti_processi:
    dataset_with_ambiti_processi[AP] = 0.0

Col codice seguente, ogni studente sarà caratterizzato da un valore per ogni ambito e processo corrispondente alla proporzione di domande che vi si riferivano e a cui ha risposto correttamente: ad esempio, nel caso un processo `x` vada a caratterizzare 10 domande e lo studente risponda correttamente a 5 di queste ultime, allora il valore che quello studente presenterà sotto il processo `x` sarà pari a $0.5$.

In [25]:
"""
Eseguire per: rimozione feature sulle domande e aggiunta feature sugli ambiti e processi
"""
questions_columns = [col for col in list(cleaned_original_dataset) if re.search("^D\d", col)]

for i, row in dataset_with_ambiti_processi.iterrows():
    for question, APs in MAPPING_DOMANDE_AMBITI_PROCESSI.items():
        if row[question] == True:   # se ha risposto correttamente
            for AP in APs:
                dataset_with_ambiti_processi.at[i, AP] += 1 / conteggio_ambiti_processi[AP]

dataset_ap = dataset_with_ambiti_processi.drop(questions_columns, axis=1)

dataset_ap.to_csv(cfg.CLEANED_DATASET_WITH_AP)

In [30]:
"""
Eseguire per: rimozione feature sulle domande e aggiunta feature sugli ambiti e processi
Attenzione: inutile eseguire se si è eseguita la precedente cella.
"""
dataset_ap = pd.read_csv(cfg.CLEANED_DATASET_WITH_AP)

In [31]:
if "Unnamed: 0" in dataset_ap.columns:
    dataset_ap.drop("Unnamed: 0", axis=1, inplace=True)

## Analisi della correlazione fra feature

In [47]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [50]:
corr_matrix = dataset_ap.corr(method='pearson').round(2)
corr_matrix.style.background_gradient(cmap='YlOrRd')

,CODICE_SCUOLA,CODICE_PLESSO,CODICE_CLASSE,campione,prog,voto_scritto_ita,voto_orale_ita,voto_scritto_mat,voto_orale_mat,Pon,pu_ma_gr,pu_ma_no,Fattore_correzione_new,Cheating,WLE_MAT,WLE_MAT_200,WLE_MAT_200_CORR,pu_ma_no_corr,n_stud_prev,n_classi_prev,LIVELLI,DROPOUT,Dati e previsioni,"Risolvere problemi utilizzando strategie in ambiti diversi – numerico, geometrico, algebrico –",Conoscere e utilizzare algoritmi e procedure,"Utilizzare strumenti, modelli e rappresentazioni nel trattamento quantitativo dell'informazione in ambito scientifico, tecnologico, economico e sociale",Numeri,"Riconoscere in contesti diversi il carattere misurabile di oggetti e fenomeni, utilizzare strumenti di misura, misurare grandezze, stimare misure di grandezze","Rappresentare relazioni e dati e, in situazioni significative, utilizzare le rappresentazioni per ricavare informazioni, formulare giudizi e prendere decisioni",Conoscere e padroneggiare i contenuti specifici della matematica,Acquisire progressivamente forme tipiche del pensiero matematico,Relazioni e funzioni,Spazio figure,Riconoscere le forme nello spazio e utilizzarle per la risoluzione di problemi geometrici o di modellizzazione
CODICE_SCUOLA,1.000000,-0.010000,1.000000,0.000000,-0.050000,-0.020000,-0.030000,-0.020000,-0.020000,-0.080000,-0.030000,-0.030000,0.040000,-0.040000,-0.010000,-0.030000,-0.000000,-0.010000,-0.160000,-0.380000,-0.030000,0.020000,-0.030000,-0.030000,-0.020000,-0.030000,-0.030000,-0.020000,-0.010000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000
CODICE_PLESSO,-0.010000,1.000000,-0.010000,-0.010000,0.010000,0.000000,0.010000,0.000000,0.010000,-0.020000,0.010000,0.010000,-0.010000,0.010000,0.000000,0.010000,0.010000,0.010000,0.020000,0.010000,0.010000,-0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
CODICE_CLASSE,1.000000,-0.010000,1.000000,0.000000,-0.050000,-0.020000,-0.030000,-0.020000,-0.020000,-0.080000,-0.030000,-0.030000,0.040000,-0.040000,-0.010000,-0.030000,-0.000000,-0.010000,-0.160000,-0.380000,-0.030000,0.020000,-0.030000,-0.030000,-0.020000,-0.030000,-0.030000,-0.020000,-0.010000,-0.020000,-0.020000,-0.020000,-0.020000,-0.020000
campione,0.000000,-0.010000,0.000000,1.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.010000,-0.040000,-0.020000,-0.020000,-0.010000,0.010000,-0.010000,-0.020000,0.040000,-0.020000,0.010000,-0.020000,-0.000000,0.000000,-0.010000,-0.020000,-0.020000,-0.010000,-0.020000,-0.010000,-0.000000,-0.010000,-0.010000,-0.010000,-0.010000,-0.010000
prog,-0.050000,0.010000,-0.050000,0.000000,1.000000,0.020000,0.020000,0.010000,0.020000,-0.000000,0.020000,0.020000,0.000000,-0.000000,0.010000,0.020000,0.020000,0.020000,0.280000,0.060000,0.020000,-0.020000,0.020000,0.020000,0.020000,0.020000,0.020000,0.010000,0.000000,0.020000,0.010000,0.010000,0.020000,0.010000
voto_scritto_ita,-0.020000,0.000000,-0.020000,-0.000000,0.020000,1.000000,0.870000,0.750000,0.660000,-0.030000,0.430000,0.430000,-0.040000,0.040000,0.060000,0.440000,0.380000,0.400000,0.050000,-0.010000,0.370000,-0.310000,0.340000,0.260000,0.360000,0.340000,0.360000,0.270000,0.150000,0.290000,0.300000,0.340000,0.340000,0.220000
voto_orale_ita,-0.030000,0.010000,-0.030000,-0.000000,0.020000,0.870000,1.000000,0.650000,0.720000,-0.030000,0.450000,0.450000,-0.020000,0.020000,0.070000,0.450000,0.410000,0.430000,0.060000,-0.000000,0.400000,-0.340000,0.350000,0.280000,0.370000,0.350000,0.370000,0.280000,0.160000,0.300000,0.320000,0.360000,0.360000,0.230000
voto_scritto_mat,-0.020000,0.000000,-0.020000,-0.000000,0.010000,0.750000,0.650000,1.000000,0.880000,-0.050000,0.520000,0.520000,-0.040000,0.040000,0.070000,0.530000,0.470000,0.490000,0.030000,-0.020000,0.480000,-0.390000,0.400000,0.340000,0.430000,0.390000,0.440000,0.330000,0.190000,0.360000,0.360000,0.410000,0.420000,0.270000
voto_orale_mat,-0.020000,0.010000,-0.020000,-0.010000,0.020000,0.660000,0.720000,0.880000,1.000000,-0.050000,0.550000,0.550000,-0.010000,0.0100

Emerge un'alta correlazione ($0.87$) tra i voti della stessa materia, come era prevedibile, mentre una correlazione abbastanza alta tra materie differenti ($0.75$). La correlazione pari a $1$ tra `pu_ma_gr` e `pu_ma_no` si spiega considerando che la seconda è la normalizzazione della prima, per cui portano esattamente la stessa informazione.

Abbiamo ritenuto interessante indagare la correlazione sussistente tra i voti agli scritti e agli orali, i punteggi finali ottenuti al test e gli ambiti e i processi: abbiamo previsto solo valori positivi di correlazione, in quanto a voti maggiori, corrispondono punteggi finali maggiori, come anche per i processi e ambiti. 

In [55]:
interesting_to_check_if_correlated_columns = [
    # Alta correlazione fra voti della stessa materia, abbastanza correlate fra materie diverse
    "voto_scritto_ita",
    "voto_orale_ita",
    "voto_scritto_mat",
    "voto_orale_mat",
    # Correlazione totale, abbastanza correlate con voti
    "pu_ma_no"
] + list(ambiti_processi)

In [56]:
check_corr_dataset = dataset_ap[interesting_to_check_if_correlated_columns].corr(method='pearson').round(2)
check_corr_dataset.style.background_gradient(cmap='YlOrRd')

,voto_scritto_ita,voto_orale_ita,voto_scritto_mat,voto_orale_mat,pu_ma_no,Dati e previsioni,"Risolvere problemi utilizzando strategie in ambiti diversi – numerico, geometrico, algebrico –",Conoscere e utilizzare algoritmi e procedure,"Utilizzare strumenti, modelli e rappresentazioni nel trattamento quantitativo dell'informazione in ambito scientifico, tecnologico, economico e sociale",Numeri,"Riconoscere in contesti diversi il carattere misurabile di oggetti e fenomeni, utilizzare strumenti di misura, misurare grandezze, stimare misure di grandezze","Rappresentare relazioni e dati e, in situazioni significative, utilizzare le rappresentazioni per ricavare informazioni, formulare giudizi e prendere decisioni",Conoscere e padroneggiare i contenuti specifici della matematica,Acquisire progressivamente forme tipiche del pensiero matematico,Relazioni e funzioni,Spazio figure,Riconoscere le forme nello spazio e utilizzarle per la risoluzione di problemi geometrici o di modellizzazione
voto_scritto_ita,1.000000,0.870000,0.750000,0.660000,0.430000,0.340000,0.260000,0.360000,0.340000,0.360000,0.270000,0.150000,0.290000,0.300000,0.340000,0.340000,0.220000
voto_orale_ita,0.870000,1.000000,0.650000,0.720000,0.450000,0.350000,0.280000,0.370000,0.350000,0.370000,0.280000,0.160000,0.300000,0.320000,0.360000,0.360000,0.230000
voto_scritto_mat,0.750000,0.650000,1.000000,0.880000,0.520000,0.400000,0.340000,0.430000,0.390000,0.440000,0.330000,0.190000,0.360000,0.360000,0.410000,0.420000,0.270000
voto_orale_mat,0.660000,0.720000,0.880000,1.000000,0.550000,0.420000,0.360000,0.450000,0.420000,0.460000,0.340000,0.200000,0.380000,0.380000,0.440000,0.440000,0.290000
pu_ma_no,0.430000,0.450000,0.520000,0.550000,1.000000,0.760000,0.670000,0.770000,0.760000,0.830000,0.620000,0.420000,0.680000,0.660000,0.800000,0.800000,0.570000
Dati e previsioni,0.340000,0.350000,0.400000,0.420000,0.760000,1.000000,0.450000,0.650000,0.870000,0.530000,0.390000,0.260000,0.450000,0.400000,0.520000,0.500000,0.330000
"Risolvere problemi utilizzando strategie in ambiti diversi – numerico, geometrico, algebrico –",0.260000,0.280000,0.340000,0.360000,0.670000,0.450000,1.000000,0.480000,0.440000,0.620000,0.390000,0.230000,0.370000,0.360000,0.450000,0.590000,0.300000
Conoscere e utilizzare algoritmi e procedure,0.360000,0.370000,0.430000,0.450000,0.770000,0.650000,0.480000,1.000000,0.540000,0.630000,0.420000,0.260000,0.440000,0.420000,0.550000,0.640000,0.340000
"Utilizzare strumenti, modelli e rappresentazioni nel trattamento quantitativo dell'informazione in ambito scientifico, tecnologico, economico e sociale",0.340000,0.350000,0.390000,0.420000,0.760000,0.870000,0.440000,0.540000,1.000000,0.520000,0.390000,0.260000,0.420000,0.410000,0.630000,0.510000,0.320000
Numeri,0.360000,0.370000,0.440000,0.460000,0.830000,0.530000,0.620000,0.630000,0.520000,1.000000,0.640000,0.260000,0.790000,0.490000,0.540000,0.530000,0.340000


La nostra previsione è stata confermata.

### Rimozione colonne altamente correlate

In [ ]:
# TODO Rimozione colonne altamente correlate

## Analisi dei tipi delle colonne

In [87]:
print("Lista colonne e tipi:")

table = BeautifulTable()
table.columns.header = ["", "Type"]

for col in dataset_ap.columns :
    table.rows.append([col, dataset_ap[col].dtypes])
        
table.columns.alignment = BeautifulTable.ALIGN_LEFT
table.set_style(BeautifulTable.STYLE_SEPARATED)
print(table)

Lista colonne e tipi:
+====================================================================+=========+
|                                                                    | Type    |
+====================================================================+=========+
| CODICE_SCUOLA                                                      | int64   |
+--------------------------------------------------------------------+---------+
| CODICE_PLESSO                                                      | int64   |
+--------------------------------------------------------------------+---------+
| CODICE_CLASSE                                                      | int64   |
+--------------------------------------------------------------------+---------+
| campione                                                           | int64   |
+--------------------------------------------------------------------+---------+
| prog                                                               | int64   |
+-----

In [90]:
continuous_features = columns_lower_ratio_null_values + \
                      ["pu_ma_gr", "pu_ma_no", "Fattore_correzione_new", "Cheating", "WLE_MAT", "WLE_MAT_200", "WLE_MAT_200_CORR",
                       "pu_ma_no_corr"] + \
                      list(ambiti_processi) # Feature sui voti, feature elencate, ambiti e processi
ordinal_features = [
    "n_stud_prev", "n_classi_prev", "LIVELLI", "voto_scritto_mat", "voto_orale_mat"
] + (["voto_scritto_ita", "voto_orale_ita"] if cfg.FILL_NAN != "remove" else []) # se si rimuovono le colonne dei voti di italiano non vanno messe tra le feature
int_categorical_features = [
    "CODICE_SCUOLA", "CODICE_PLESSO", "CODICE_CLASSE", "campione", "prog",
]
str_categorical_features = [
    "sesso", "mese", "anno", "luogo", "eta", "freq_asilo_nido", "freq_scuola_materna",
    "luogo_padre", "titolo_padre", "prof_padre", "luogo_madre", "titolo_madre", "prof_madre",
    "regolarità", "cittadinanza", "cod_provincia_ISTAT", "Nome_reg",
    "Cod_reg", "Areageo_3", "Areageo_4", "Areageo_5", "Areageo_5_Istat"
]
bool_features = ["Pon"]

## Gestione dei valori nulli
Il dataset in considerazione presenta molti valori nulli.\
Vi sono diverse tecniche per gestirli, tra cui:
- sostituzione del valore nullo con un indice di sintesi (media, mediana, moda) della colonna in considerazione: la scelta dell'indice dipende da vari fattori, tra cui la forma della distribuzione del certo attributo;
- rimozione della riga corrispondente: viene adottata solitamente quando la colonna presenta pochi valori nulli e/o la riga presenta molti valori nulli;
- rimozione della colonna corrispondente: viene adottata solitamente quando la colonna presenta un alto numero di valori nulli.

In [91]:
dataset_ap["sigla_provincia_istat"].fillna(value="ND", inplace=True)
# TODO Trovare un modo migliore per effettuare il rimpiazzo dei non disponibili.
dataset_ap["sigla_provincia_istat"].fillna(value="ND", inplace=True)
if cfg.FILL_NAN == "median":
    dataset_ap["voto_scritto_ita"].fillna(value=dataset_ap["voto_scritto_ita"].median(), inplace=True)
    dataset_ap["voto_orale_ita"].fillna(value=dataset_ap["voto_orale_ita"].median(), inplace=True)
    dataset_ap["voto_scritto_mat"].fillna(value=dataset_ap["voto_scritto_mat"].median(), inplace=True)
    dataset_ap["voto_orale_mat"].fillna(value=dataset_ap["voto_orale_mat"].median(), inplace=True)
elif cfg.FILL_NAN == "mean":
    dataset_ap["voto_scritto_ita"].fillna(value=dataset_ap["voto_scritto_ita"].mean(), inplace=True)
    dataset_ap["voto_orale_ita"].fillna(value=dataset_ap["voto_orale_ita"].mean(), inplace=True)
    dataset_ap["voto_scritto_mat"].fillna(value=dataset_ap["voto_scritto_mat"].mean(), inplace=True)
    dataset_ap["voto_orale_mat"].fillna(value=dataset_ap["voto_orale_mat"].mean(), inplace=True)
elif cfg.FILL_NAN == "mode":
    dataset_ap["voto_scritto_ita"].fillna(value=dataset_ap["voto_scritto_ita"].mode(), inplace=True)
    dataset_ap["voto_orale_ita"].fillna(value=dataset_ap["voto_orale_ita"].mode(), inplace=True)
    dataset_ap["voto_scritto_mat"].fillna(value=dataset_ap["voto_scritto_mat"].mode(), inplace=True)
    dataset_ap["voto_orale_mat"].fillna(value=dataset_ap["voto_orale_mat"].mode(), inplace=True)
elif cfg.FILL_NAN == "remove":
    # Rimuovere colonne voti ita
    # Rimuovere record con dati nulli in voti mat
    dataset_ap.drop(["voto_scritto_ita", "voto_orale_ita"], axis=1, inplace=True)
    dataset_ap.dropna(["voto_scritto_mat", "voto_orale_mat"], inplace=True)

# Machine Learning
## Suddivisione dataset in training e test

In [92]:
df_training_set, df_test_set = train_test_split(dataset_ap, test_size=cfg.TEST_SET_PERCENT)

## Analisi e gestione dello sbilanciamento del dataset

In [117]:
nr_nodrop, nr_drop = np.bincount(dataset_ap['DROPOUT'])
total_records = nr_drop + nr_nodrop
nl = '\n'
print(
    f"Total number of records: {total_records}{nl}\
        {nl}\
Total num. DROPOUT: {nr_drop}{nl}\
Total num. NO DROPOUT: {nr_nodrop}{nl}\
    {nl}\
Ratio DROPOUT/TOTAL: {round(nr_drop / total_records, 2)}{nl}\
Ratio NO DROPOUT/TOTAL: {round(nr_nodrop / total_records, 2)}{nl}\
    {nl}\
Ratio DROPOUT/NO DROPOUT: {round(nr_drop / nr_nodrop, 2)}"
)

Total number of records: 342226
        
Total num. DROPOUT: 116782
Total num. NO DROPOUT: 225444
    
Ratio DROPOUT/TOTAL: 0.34
Ratio NO DROPOUT/TOTAL: 0.66
    
Ratio DROPOUT/NO DROPOUT: 0.52


### Campionamento: Random undersampling o SMOTE

In [93]:
"""
Eseguire per: campionamento
"""
if cfg.SAMPLING_TO_PERFORM == "random_undersampling":
    # class_nodrop contiene i record della classe sovrarappresentata, ovvero SENZA DROPOUT.
    class_nodrop = df_training_set[df_training_set['DROPOUT'] == False]
    # class_drop contiene i record della classe sottorappresentata, ovvero CON DROPOUT.
    class_drop = df_training_set[df_training_set['DROPOUT'] == True]

    # Sotto campionamento di class_drop in modo che abbia stessa cardinalità di class_nodrop
    class_nodrop = class_nodrop.sample(len(class_drop))

    print(f'Class NO DROPOUT: {len(class_nodrop):,}')
    print(f'Classe DROPOUT: {len(class_drop):,}')

    df_training_set = class_drop.append(class_nodrop)
    df_training_set = df_training_set.sample(frac=1)
elif cfg.SAMPLING_TO_PERFORM == "SMOTE":
    print("SMOTE not yet implemented")
else:
    print(f"SAMPLING_TO_PERFORM = {cfg.SAMPLING_TO_PERFORM} not recognized.")

Class NO DROPOUT: 93,341
Classe DROPOUT: 93,341


In [ ]:
if "Unnamed: 0" in df_training_set.columns:
    df_training_set.drop("Unnamed: 0", axis=1, inplace=True)

## Preprocessing per creazione del modello di Deep Learning

### Suddivisione del dataset in training e test

In [ ]:
df_training_set, df_validation_set = train_test_split(df_training_set, test_size=cfg.VALIDATION_SET_PERCENT)

### Conversione dei dati da DataFrame (Pandas) a Dataset (Tensorflow/Keras)

In [ ]:
def pd_dataframe_to_tf_dataset(dataframe: pd.DataFrame):
    copied_df = dataframe.copy()
    dropout_col = copied_df.pop("DROPOUT")
    """
    Dato che il dataframe ha dati eterogenei lo convertiamo a dizionario,
    in cui le chiavi sono i nomi delle colonne e i valori sono i valori della colonna.
    Infine bisogna indicare la colonna target.
    """
    tf_dataset = tf.data.Dataset.from_tensor_slices((dict(copied_df), dropout_col))
    tf_dataset = tf_dataset.shuffle(buffer_size=len(copied_df))
    return tf_dataset

In [ ]:
ds_training_set = pd_dataframe_to_tf_dataset(df_training_set)
ds_validation_set = pd_dataframe_to_tf_dataset(df_validation_set)
ds_test_set = pd_dataframe_to_tf_dataset(df_test_set)

### Suddivisione del dataset in batch

In [ ]:
ds_training_set = ds_training_set.batch(cfg.BATCH_SIZE, drop_remainder=True)
ds_validation_set = ds_validation_set.batch(cfg.BATCH_SIZE, drop_remainder=True)
ds_test_set = ds_test_set.batch(cfg.BATCH_SIZE, drop_remainder=True)

### Creazione dei layer di input per ogni feature

In [ ]:
input_layers = {}
for name, column in df_training_set.items():
    if name == "DROPOUT":
        continue

    if name in continuous_features:
        dtype = tf.float32
    elif name in ordinal_features or name in int_categorical_features or name in bool_features:
        dtype = tf.int64
    else:  # str_categorical_features
        dtype = tf.string

    input_layers[name] = tf.keras.Input(shape=(), name=name, dtype=dtype)

### Encoding delle feature in base al loro tipo

In [ ]:
preprocessed_features = []

def stack_dict(inputs, fun=tf.stack):
    values = []
    for key in sorted(inputs.keys()):
        values.append(tf.cast(inputs[key], tf.float32))

    return fun(values, axis=-1)

In [ ]:
# Preprocessing colonne con dati booleani
for name in bool_features:
    inp = input_layers[name]
    inp = inp[:, tf.newaxis]
    float_value = tf.cast(inp, tf.float32)
    preprocessed_features.append(float_value)

In [ ]:
# Preprocessing colonne con dati interi ordinali
ordinal_inputs = {}
for name in ordinal_features:
    ordinal_inputs[name] = input_layers[name]

normalizer = Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(df_training_set[ordinal_features])))
ordinal_inputs = stack_dict(ordinal_inputs)
ordinal_normalized = normalizer(ordinal_inputs)
preprocessed_features.append(ordinal_normalized)

In [ ]:
# Preprocessing colonne con dati continui
continuous_inputs = {}
for name in continuous_features:
    continuous_inputs[name] = input_layers[name]

normalizer = Normalization(axis=-1)
normalizer.adapt(stack_dict(dict(df_training_set[continuous_features])))
continuous_inputs = stack_dict(continuous_inputs)
continuous_normalized = normalizer(continuous_inputs)
preprocessed_features.append(continuous_normalized)

In [ ]:
# Preprocessing colonne con dati categorici stringa
for name in str_categorical_features:
    vocab = sorted(set(df_training_set[name]))

    lookup = StringLookup(vocabulary=vocab, output_mode='one_hot')

    x = input_layers[name][:, tf.newaxis]
    x = lookup(x)

    preprocessed_features.append(x)

In [ ]:
# Preprocessing colonne con dati categorici interi
for name in int_categorical_features:
    vocab = sorted(set(df_training_set[name]))

    lookup = IntegerLookup(vocabulary=vocab, output_mode='one_hot')

    x = input_layers[name][:, tf.newaxis]
    x = lookup(x)

    preprocessed_features.append(x)

### Assemblaggio dei vari layer e creazione del modello

In [ ]:
# initializer = tf.keras.initializers.glorot_uniform(seed=19)

preprocessed = tf.concat(preprocessed_features, axis=-1)

preprocessor = tf.keras.Model(input_layers, preprocessed)

"""
body = tf.keras.Sequential(
    [tf.keras.layers.Dense(cfg.NEURONS, activation=cfg.DENSE_LAYER_ACTIVATION) for _ in range(cfg.NUMBER_OF_LAYERS)] +
    [tf.keras.layers.Dropout(cfg.DROPOUT_LAYER_RATE)] if cfg.DROPOUT_LAYER else [] +
    [tf.keras.layers.Dense(1, activation=cfg.OUTPUT_ACTIVATION_FUNCTION)]
)
"""
body = tf.keras.Sequential()
for _ in range(cfg.NUMBER_OF_LAYERS):
    if cfg.LEAKY_RELU:
        body.add(tf.keras.layers.Dense(cfg.NEURONS))
        body.add(tf.keras.layers.LeakyReLU())
    else:
        body.add(tf.keras.layers.Dense(cfg.NEURONS, activation=cfg.DENSE_LAYER_ACTIVATION))
if cfg.DROPOUT_LAYER:
    body.add(tf.keras.layers.Dropout(cfg.DROPOUT_LAYER_RATE))
body.add(tf.keras.layers.Dense(1, activation=cfg.OUTPUT_ACTIVATION_FUNCTION))

x = preprocessor(input_layers)

result = body(x)

model = tf.keras.Model(input_layers, result)

In [ ]:
# Visualizzazione tabellare del modello
from keras.utils.vis_utils import plot_model
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

### Compilazione del modello e scelta delle metriche

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=cfg.LEARNING_RATE),
              loss=tf.losses.BinaryCrossentropy(),
              metrics=[
                  tf.metrics.Accuracy(),
                  tf.metrics.BinaryAccuracy(),
                  tf.metrics.Precision(),
                  tf.metrics.Recall(),
                  tf.metrics.FalseNegatives(),
                  tf.metrics.FalsePositives(),
                  tf.metrics.TrueNegatives(),
                  tf.metrics.TruePositives()])

### Early stopping

In [ ]:
"""
Definizione dello stopper per evitare che la reti continui ad addestrarsi quando non ci sono miglioramenti della loss 
(val_loss = funzione di costo sul validation set) per piu' di 3 epoche
"""
early_stopper = EarlyStopping(monitor="val_loss", patience=2)

### Training

In [ ]:
model.fit(ds_training_set,
          epochs=cfg.EPOCH,
          batch_size=cfg.BATCH_SIZE,
          validation_data=ds_validation_set,
          callbacks=[early_stopper],
          verbose=2)

### Test

In [ ]:
score = model.evaluate(ds_test_set)

## Risultati ottenuti


| Epoche | Neuroni | Batch | Tecnica di sampling | Layer lineari | Tasso apprendimento | Dropout | Tasso di dropout | Accuratezza in training | Accuratezza in test | Note aggiuntive |
| :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |
|     10 |     128 |    32 |       Undersampling |            10 |               0.001 |      No |                - |                  0.2339 |              0.2336 | C'erano colonne con alta correlazione. |
|     50 |     128 |    32 |       Undersampling |            10 |               0.001 |      No |                - |                  0.6938 |              0.6874 | C'erano colonne con alta correlazione. |
|     50 |     256 |    32 |       Undersampling |            10 |               0.001 |      No |                - |                  0.6944 |              0.6873 | C'erano colonne con alta correlazione. |
|     50 |     128 |   128 |       Undersampling |            10 |               0.001 |      No |                - |                  0.2386 |              0.2341 | C'erano colonne con alta correlazione. Perché è così pessimo il risultato? |

Per info su come formattare Markdown: [clicca qui](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd).